<h1>Importing Libraries</h1>

In [1]:
import pandas as pd
import json
import numpy as np
from google.cloud import bigquery
import hashlib

<strong>Requirements.txt</strong> <br>
pandas <br>
numpy <br>
google-cloud-bigquery <br>
fsspec <br>
gcsfs <br>
pyarrow <br>
pandas-gbq <br>

<h1>Event Trigger function in Cloud Functions</h1>

In [ ]:
def hello_gcs(event, context):
    """Triggered by a change to a Cloud Storage bucket.
    Args:
    event (dict): Event payload.
    context (google.cloud.functions.Context): Metadata for the event.
    """
    file = event
    #print(f"Processing file: {file['name']}.")
    path = file['name']
    INPUT_PATH = f"gs://{file['bucket']}/{path}"
    print(INPUT_PATH)
    #return INPUT_PATH
    if ('customer' in INPUT_PATH):
        OUTPUT_PATH = 'acn-in-cf-data-ggl-aca-c01-t04.test1.func_scale_customer_test1'
        return ingest_customer_data(INPUT_PATH, OUTPUT_PATH)
    elif ('CDR' in INPUT_PATH):
        OUTPUT_PATH = 'acn-in-cf-data-ggl-aca-c01-t04.test1.func_scale_CDR_test1'
        return ingest_CDR_data(INPUT_PATH, OUTPUT_PATH)
    elif ('internet' in INPUT_PATH):
        OUTPUT_PATH = 'acn-in-cf-data-ggl-aca-c01-t04.test1.func_scale_internet_test1'
        return ingest_internet_data(INPUT_PATH, OUTPUT_PATH)
    elif ('transaction' in INPUT_PATH):
        OUTPUT_PATH = 'acn-in-cf-data-ggl-aca-c01-t04.test1.func_scale_trans_test1'
        return ingest_trans_data(INPUT_PATH, OUTPUT_PATH)
    else:
        return 'ERROR'
    return 'SUCCESS'


<h1>Function for Customer Data Ingestion</h1>

In [ ]:
def ingest_customer_data(INPUT_PATH, OUTPUT_PATH):
    data = pd.read_json(INPUT_PATH, lines=True)
    df = pd.DataFrame()
    customer_name1 = []
    customer_id1 = []
    Gender = []
    zip_code = []
    connection_type = []
    connection_plan = []
    new_ported = []
    old_operator = []
    curr_operator = []
    phone_number = []
    contract_date = []
    SSN = []
    DL = []
    for lab, row in data.iterrows():
        customer_name1.append(row['root']['cus_name'])
        customer_id1.append(row['root']['cusomer_id'])
        Gender.append(row['root']['gender'])
        connection_type.append(row['connection']['type'])
        connection_plan.append(row['connection']['plan'])
        zip_code.append(row['identities']['DL']['address']['zip_code'])
        new_ported.append(row['phone']['new/port']['ported'])
        old_operator.append(row['phone']['new/port']['old_operator'])
        phone_number.append(row['phone']['number'])
        curr_operator.append(row['phone']['operator'])
        contract_date.append(row['phone']['start_date'])
        SSN.append(row['identities']['SSN']['number'])
        DL.append(row['identities']['DL']['number'])
   

In [2]:
data = pd.read_json("gs://in-telecom-t4/Telecom/customer_data.json", lines=True)

In [3]:
data.head()

,root,identities,seller,connection,phone,other_phone
0,"{'cus_name': 'Heather Johnson', 'cusomer_id': ...","{'SSN': {'number': '106-12-5365'}, 'DL': {'num...","{'name': 'Jamie Scott', 'address': {'home': '5...","{'plan': 'Both', 'type': 'Prepaid'}","{'number': '5034147053569', 'operator': 'P-Mob...","{'number': 2034378496853, 'operator': 'Bird PLC'}"
1,"{'cus_name': 'Dustin Romero', 'cusomer_id': 'd...","{'SSN': {'number': '858-81-1906'}, 'DL': {'num...","{'name': 'Kaitlyn Knight', 'address': {'home':...","{'plan': 'Data only', 'type': 'Prepaid'}","{'number': 7613062600667, 'operator': 'P-Mobil...","{'number': 5614253555384, 'operator': 'Bird PLC'}"
2,"{'cus_name': 'Keith Rodriguez', 'cusomer_id': ...","{'SSN': {'number': '680-85-8589'}, 'DL': {'num...","{'name': 'Alexandra Blackwell', 'address': {'h...","{'plan': 'Both', 'type': 'Prepaid'}","{'number': '3978831593479', 'operator': 'P-Mob...","{'number': 4353033341134, 'operator': 'Cole-Mo..."
3,"{'cus_name': 'Ashley Watson', 'cusomer_id': '2...","{'SSN': {'number': '931-32-1199'}, 'DL': {'num...","{'name': 'Michael Smith', 'address': {'home': ...","{'plan': 'Both', 'type': 'Post-paid'}","{'number': '2359928553481', 'operator': 'P-Mob...","{'number': 8696190152988, 'operator': 'Bird PLC'}"
4,"{'cus_name': 'John Simmons', 'cusomer_id': 'a8...","{'SSN': {'number': '191-23-9845'}, 'DL': {'num...","{'name': 'Brianna Jarvis', 'address': {'home':...","{'plan': 'Both', 'type': 'Prepaid'}","{'number': '5427481149458', 'operator': 'P-Mob...","{'number': 1795999414788, 'operator': 'Cole-Mo..."


In [4]:
for cols in data.columns:
    print(data[cols][0])

{'cus_name': 'Heather Johnson', 'cusomer_id': '028146e6-edc8-4b4e-8d87-69fc952d4f01', 'marital_status': 'Married', 'gender': 'Female', 'DOB': '19840105', 'occupation': 'Information Technology', 'address': {'cus_home': 'Unit 5824 Box 8279', 'cus_city': 'DPO AE 03217', 'cus_zipcode': '03217', 'cus_state': 'AE'}}
{'SSN': {'number': '106-12-5365'}, 'DL': {'number': 'UY1814444734234', 'address': {'home': 'Unit 5824 Box 8279', 'city': 'DPO AE 03217', 'state': 'AE', 'zip_code': '03217'}}}
{'name': 'Jamie Scott', 'address': {'home': '51348 Baker Glens', 'city': 'Lake Deborahchester', 'state': 'TN', 'zipcode': '61108'}}
{'plan': 'Both', 'type': 'Prepaid'}
{'number': '5034147053569', 'operator': 'P-Mobile', 'start_date': '20141122', 'new/port': {'ported': True, 'old_operator': 'Martinez LLC'}}
{'number': 2034378496853, 'operator': 'Bird PLC'}


In [ ]:
def mask(df, OUTPUT_PATH):   
   df['customer_id'] = customer_id1
   df['customer_name'] = customer_name1
   df['phone_number'] = phone_number
   df['zip_code'] = zip_code
   df['Gender'] = Gender
   df['connection_type'] = connection_type
   df['connection_plan'] = connection_plan
   df['new_ported'] = new_ported
   df['old_operator'] = old_operator
   df['curr_operator'] = curr_operator
   df['contract_date'] = contract_date
   df['SSN'] = SSN
   df['DL'] = DL
   df['SSN_hashed'] = df['SSN'].apply(lambda x: hashlib.sha256(x.encode()).hexdigest())
   df['DL_hashed'] = df['DL'].apply(lambda x: hashlib.sha256(x.encode()).hexdigest())
   df.drop(['SSN', 'DL'], axis = 1, inplace = True)
   df.to_gbq(OUTPUT_PATH,  project_id='acn-in-cf-data-ggl-aca-c01-t04', if_exists='replace')
   #to_bq(df, OUTPUT_PATH)
   return "SUCCESS customer data"

<h2>Key Points about this function</h2>
<ul>
    <li><strong>Masking the data: </strong>PII data present here is SSN (Social Security Number) number and DL number. Using the <i>hashlib</i> inbuilt python library. This uses <U><B>SHA (Secure Hashing Algorithms)</B></U> which are a set of cryptographic hash functions. <br>
        The PII data has been masked using <b><u>sha25</u></b> hashing algorithm, which is a part of hashlib library. <b><u>encode()</u></b> encodes the given string to UTF-8. <b><u>hexdigest()</u></b> returns the encoded data in a hexadecimal string format. </li>
    <li><strong>Ingesting data: </strong>Using pandas-gbq package's df.to_gb1() function for this purpose</li>

<h1>Function for CDR Data Ingestion</h1>

In [ ]:
def ingest_CDR_data(INPUT_PATH, OUTPUT_PATH):
    data = pd.read_json(INPUT_PATH, lines=True)
    source_no = []
    dest_no = []
    start_time = []
    duration = []
    source_operator = []
    dest_operator = []
    source_location = []
    dest_location = []
    disposition = []
    for lab, row in data.iterrows():
        source_no.append(row['source']['msisdn'])
        dest_no.append(row['destination']['msisdn'])
        start_time.append(row['root']['start_time'])
        duration.append(float(row['root']['duration']))
        source_operator.append(row['source']['operator'])
        dest_operator.append(row['destination']['operator'])
        source_location.append(row['source']['location'])
        dest_location.append(row['destination']['location'])
        disposition.append(row['root']['disposition'])
    df = pd.DataFrame()
    df['source_no'] = source_no
    df['dest_no'] = dest_no
    df['start_time1'] = start_time
    df['start_time'] = pd.to_datetime(df['start_time1'])
    df.drop('start_time1', axis = 1, inplace = True)
    df['duration'] = duration
    df['source_operator'] = source_operator
    df['dest_operator'] = dest_operator
    df['source_location'] = source_location
    df['dest_location'] = dest_location
    df['disposition'] = disposition
    df.replace('nan', np.nan, inplace=True)
    #df.to_gbq(OUTPUT_PATH,  project_id='acn-in-cf-data-ggl-aca-c01-t04', if_exists='append')
    to_bq(df, OUTPUT_PATH)
    return "SUCCESS CDR data"

<h1>Function for Internet Data Ingestion</h1>

In [1]:
def ingest_internet_data(INPUT_PATH, OUTPUT_PATH):
    data = pd.read_json(INPUT_PATH, lines=True)
    phone_no = []
    source_IP = []
    dest_IP = []
    start_time = []
    source_port = []
    dest_port = []
    ASN = []
    no_of_packets = []
    no_of_bytes = []
    for lab, row in data.iterrows():
        phone_no.append(row['root']['phone_number'])
        source_IP.append(row['source']['ip'])
        dest_IP.append(row['destination']['ip'])
        start_time.append(row['root']['time'])
        source_port.append(row['source']['port'])
        dest_port.append(row['destination']['port'])
        ASN.append(row['root']['asn'])
        no_of_packets.append(float(row['root']['num_packets']))
        no_of_bytes.append(float(row['root']['num_bytes']))
    df = pd.DataFrame()
    df['phone_no'] = phone_no
    df['source_IP'] = source_IP
    df['dest_IP'] = dest_IP
    df['start_time1'] = start_time
    df['start_time'] = pd.to_datetime(df['start_time1'])
    df.drop('start_time1', axis = 1, inplace = True)
    df['source_port'] = source_port
    df['dest_port'] = dest_port
    df['ASN'] = ASN
    df['no_of_packets'] = no_of_packets
    df['no_of_bytes'] = no_of_bytes
    df.replace('nan', np.nan, inplace=True)
    #df.to_gbq(OUTPUT_PATH,  project_id='acn-in-cf-data-ggl-aca-c01-t04', if_exists='append')
    to_bq(df, OUTPUT_PATH)
    return "SUCCESS internet data"

<h1>Call for Testing the function</h1>

In [63]:
event = {"bucket":"test-ingestion100", "name":"internet/internet_data_json_2021-04-01_00:00:00.json"}
hello_gcs(event, "hello")

gs://test-ingestion100/internet/internet_data_json_2021-04-01_00:00:00.json


1it [00:09,  9.03s/it]


'SUCCESS internet data'

<h1>Function for Loading Data Intro BigQuery</h1>

In [10]:
def to_bq(df, TABLE_ID):
    client = bigquery.Client()
    job_config = bigquery.LoadJobConfig(write_disposition='WRITE_TRUNCATE', autodetect = True)
    load_job = client.load_table_from_dataframe(df, table_id, job_config=job_config)  # Make an API request.
    #load_job.result()
    return 'SUCCESS'

In [ ]:
client = bigquery.Client()
table_id = 'acn-in-cf-data-ggl-aca-c01-t04.test1.CDR_special1'
job_config = bigquery.LoadJobConfig(write_disposition='WRITE_TRUNCATE', autodetect = True)

load_job = client.load_table_from_dataframe(df, table_id, job_config=job_config)  # Make an API request.
print('test')
load_job.result()
print("SUCCESS ")

In [50]:
if('customer' in INPUT_PATH):
    print("YES")
else: print("NO")

NO


In [66]:
!pip3 install fsspec

In [70]:
!pip3 install gcsfs

In [93]:
def ingest_CDR_data(INPUT_PATH):
    data = pd.read_json(INPUT_PATH, lines=True)
    source_no = []
    dest_no = []
    start_time = []
    duration = []
    source_operator = []
    dest_operator = []
    source_location = []
    dest_location = []
    disposition = []
    for lab, row in data.iterrows():
        source_no.append(row['source']['msisdn'])
        dest_no.append(row['destination']['msisdn'])
        start_time.append(row['root']['start_time'])
        duration.append(float(row['root']['duration']))
        source_operator.append(row['source']['operator'])
        dest_operator.append(row['destination']['operator'])
        source_location.append(row['source']['location'])
        dest_location.append(row['destination']['location'])
        disposition.append(row['root']['disposition'])
    df = pd.DataFrame()
    df['source_no'] = source_no
    df['dest_no'] = dest_no
    df['start_time1'] = start_time
    df['start_time'] = pd.to_datetime(df['start_time1'])
    df.drop('start_time1', axis = 1, inplace = True)
    df['duration'] = duration
    df['source_operator'] = source_operator
    df['dest_operator'] = dest_operator
    df['source_location'] = source_location
    df['dest_location'] = dest_location
    df['disposition'] = disposition
    df.replace('nan', np.nan, inplace=True)
    #df.to_gbq(OUTPUT_PATH,  project_id='acn-in-cf-data-ggl-aca-c01-t04', if_exists='append')
    client = bigquery.Client(project='acn-in-cf-data-ggl-aca-c01-t04')
    table_id = 'test1.CDR_special'
    job_config = bigquery.LoadJobConfig(autodetect=True, write_disposition=bigquery.WriteDisposition.WRITE_APPEND)
    try:
        load_job = client.load_table_from_dataframe(df, table_id, job_config=job_config)  # Make an API request.
        job.result()
        return 'SUCCESS CDR'
    except:
        return 'ERROR'
    #return "SUCCESS CDR data"

In [95]:
df.head()

NameError: name 'df' is not defined

In [86]:
def df_to_bq(df, table_id, client):
    job_config = bigquery.LoadJobConfig(autodetect=True, write_disposition=bigquery.WriteDisposition.WRITE_APPEND)
    #table = get_bigquery_table(table_id, dataset_id, client)
    try:
        load_job = client.load_table_from_dataframe(df, table_id, job_config=job_config)  # Make an API request.
        load_job.result()  # Waits for the job to complete.
        # if load_job.errors:
        #     logging.info(f"result={load_job.error_result}")
        #     logging.info(f"errors = {load_job.errors}")
        # else:
        #     logging.info(f'Loaded {len(df)} rows.')

    except Exception as error:
        return 'Error'

    # if load_job.errors:
    #     logging.info(f"result={load_job.error_result}")
    #     logging.info(f"errors={load_job.errors}")
    #return table
    return 'SUCCESS'

In [94]:
ingest_CDR_data('gs://test-ingestion100/CDR/data_json_2021-04-01_00:00:00.json')

'ERROR'

In [15]:
!pip3 install pyarrow